In [ ]:
!pip install sympy numpy pandas matplotlib scipy ipython plotly nbformat
# this should take up to 7 minutes

In [41]:
import warnings
# Ignore SymPyDeprecationWarning
warnings.filterwarnings("ignore")
from sympy import *
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from scipy.integrate import odeint
from scipy.optimize import fsolve
from IPython.display import display, Math, Latex
from plotly import graph_objects as go

In [ ]:

def model_simulator(y, t, model_dictionary):
    for time, cur_model in model_dictionary.items():
        if time[0] <= t < time[1]:
            return cur_model['model'](*y).flatten()


def model_fixed_point_function(ic, model_function):
    return model_function(*ic).flatten()


def simulate_HPA(ic, sim_time, model_dictionary, variables):
    result = odeint(model_simulator, ic, sim_time, args=(model_dictionary,), hmax=0.01)
    result_df = pd.DataFrame(result, columns=variables)
   parameters = {
    u: 1,
    a1: 1,
    a2: 1,
    a3: 1,
    a4: 1,
    a5: 1,
    b1: 1,
    b2: 1,
    b3: 1,
    b4: 1,
    b5: 1,
    K_GR: 4,
    H1: 1,
    H2: 1,
    aA: 1,
    aP: 1,
    bA: 1,
    bP: 1,
    # P: 1,
    # A: 1,
    # x1: 1,
    # x2: 1,
    # x3: 1,
    # x4: 1,
    # x5: 1,
}

parameter_values_normal = {
    u: 1,
    
    b1: 0.17 * 60 * 24,
    b2: 0.035 * 60 * 24,
    b3: 0.0086 * 60 * 24,
    b4: 0.0086 * 60 * 24,
    b5: 0.0086 * 60 * 24,
    bP: 0.049,
    bA: 0.099,
    K_GR: 4,
    a1: 0.17 * 60 * 24,
    a2: 0.035 * 60 * 24,
    a3: 0.0086 * 60 * 24,
    a4: 0.0086 * 60 * 24,
    a5: 0.0086 * 60 * 24,
    aP: 0.049,
    aA: 0.099,
} return result_df

In [ ]:
def crate_symbol(name):
    return symbols(name)#, real=True, positive=True)

u = crate_symbol("u")

x1 = crate_symbol("x1")
x2 = crate_symbol("x2")
x3 = crate_symbol("x3")
x4 = crate_symbol("x4")
x5 = crate_symbol("x5")

a1 = crate_symbol("a1")
a2 = crate_symbol("a2")
a3 = crate_symbol("a3")
a4 = crate_symbol("a4")
a5 = crate_symbol("a5")

b1 = crate_symbol("b1")
b2 = crate_symbol("b2")
b3 = crate_symbol("b3")
b4 = crate_symbol("b4")
b5 = crate_symbol("b5")

K_GR = crate_symbol("K_GR")
H1 = crate_symbol("H1")
H2 = crate_symbol("H2")
P = crate_symbol("P")
A = crate_symbol("A")
t = crate_symbol("t")

aP = crate_symbol("aP")
aA = crate_symbol("aA")

bP = crate_symbol("bP")
bA = crate_symbol("bA")

mr = 1 / x3
gr = K_GR ** 3 / (x3 ** 3 + K_GR ** 3)
f = gr * mr
dx1_dt = b1 * u * gr * mr * x5 - a1 * x1
dx2_dt = b2 * x1 * P * gr * x5- a2 * x2
dx3_dt = b3 * x2 * A *x4 * x5 - a3 * x3
dx4_dt = b4 * u -  a4 * x4
dx5_dt = b5 * u * gr - a5 * x5
dP_dt = P * (bP * x1 * x4 - aP)
dA_dt = A * (bA * x2 - aA)

display(
    Matrix(
        [
            Eq(Derivative(x1, t), dx1_dt),
            Eq(Derivative(x2, t), dx2_dt),
            Eq(Derivative(x3, t), dx3_dt),
            Eq(Derivative(x4, t), dx4_dt),
            Eq(Derivative(x5, t), dx5_dt),
            Eq(Derivative(P, t), dP_dt),
            Eq(Derivative(A, t), dA_dt),
        ]
    )
)
# ignore the warning
model = Matrix([dx1_dt, dx2_dt, dx3_dt, dx4_dt, dx5_dt, dP_dt, dA_dt]) 
variables = (x1, x2, x3, x4, x5, P, A)

Matrix([
[Eq(Derivative(x1, t), K_GR**3*b1*u*x5/(x3*(K_GR**3 + x3**3)) - a1*x1)],
[  Eq(Derivative(x2, t), K_GR**3*P*b2*x1*x5/(K_GR**3 + x3**3) - a2*x2)],
[                         Eq(Derivative(x3, t), A*b3*x2*x4*x5 - a3*x3)],
[                                 Eq(Derivative(x4, t), -a4*x4 + b4*u)],
[        Eq(Derivative(x5, t), K_GR**3*b5*u/(K_GR**3 + x3**3) - a5*x5)],
[                             Eq(Derivative(P, t), P*(-aP + bP*x1*x4))],
[                                Eq(Derivative(A, t), A*(-aA + bA*x2))]])

In [36]:
parameter_values_sim_high = parameter_values_normal.copy()
parameter_values_sim_low = parameter_values_normal.copy()
parameter_values_sim_high[u] = 2

model_high_u = lambdify(variables, model.subs(parameter_values_sim_high))
model_low_u = lambdify(variables, model.subs(parameter_values_sim_low))
parameter_values_sim_high[K_GR] =2
model_high_u_low_kgr = lambdify(variables, model.subs(parameter_values_sim_high))
parameter_values_sim_low[K_GR] = 2
model_low_u_low_kgr = lambdify(variables, model.subs(parameter_values_sim_low))

guess = [1 for _ in range(len(variables))]

low = {'model': model_low_u, 'params': parameter_values_sim_low}
high = {'model': model_high_u, 'params': parameter_values_sim_high}
low_kgr = {'model': model_low_u_low_kgr, 'params': parameter_values_sim_low}
high_kgr = {'model': model_high_u_low_kgr, 'params': parameter_values_sim_high}

day = 1
hour = day / 24
year = 365 * day
L = 100 * day
dt = hour
sim_time = np.arange(0, L, dt)

model_dictionary = {  # specify which model to use in each time interval 
    (5 *day, 8 * day): high,
    (8 * day, 10 * day): low,
    (10 * day, 20* day): low_kgr,
    (20 * day, 25 * day): high_kgr,
    (25 * day, np.inf* day): low_kgr,
    (-np.inf, np.inf): low
}

ic = fsolve(model_fixed_point_function, guess, args=model_low_u)

res = simulate_HPA(ic, sim_time, model_dictionary,variables)
time_scale = 'days'
if time_scale == 'hours':
    res['t'] = sim_time * 24
else:
    res['t'] = sim_time
res = res.set_index('t')
res.plot(backend='plotly').show()
plt.tight_layout()
# plt.xticks(np.arange(0, L, day), np.arange(0, L, 2 * day))
plt.show()

<Figure size 640x480 with 0 Axes>